In [7]:
import pandas as pd
import pymongo
import numpy as np
import gzip

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [ ]:
to_insert = []
for i,row in enumerate(gzip.open("../data/Uniprot/idmapping_selected.tab.gz","r")):
    r = pd.Series(row.decode("utf-8").split("\t"))[[0,1,2,9,12,15,21]].copy()
    r.index = ["uniprot_acc","uniprot_id","gene_id","uniref50","tax_id","pmid","pmid2"]
    r.pmid = [int(x) for x in r.pmid.split(";")] if r.pmid != "" else []
    r.pmid += [int(y) for y in r.pmid2.strip().split(";")] if r.pmid2.strip() != "" else []
    r.drop("pmid2",inplace=True)
    if len(r.pmid)>0 and r.gene_id != "":
        for g in r.gene_id.split(";"):
            for pmid in r.pmid:
                s = pd.Series([r.uniprot_acc,r.uniprot_id,int(g),r.uniref50,int(r.tax_id),int(pmid)],r.index)
                to_insert.append(dict(s))
                if len(to_insert)%25000==0:
                    db.uniprot.insert_many(to_insert)
                    to_insert = []
db.uniprot.insert_many(to_insert)